## 1. 학습모델 저장하고 재사용하기
- 포유류와 조류를 구분하는 신경망 모델을 이용
- 데이터를 csv파일로 분리 뒤 해당 파일을 읽어들여 사용하기

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
x_data=[[0,0],[1,0],[1,1],[0,0],[0,0],[0,1]]
y_data=[[1,0,0],[0,1,0],[0,0,1],[1,0,0],[1,0,0],[0,0,1]]

In [4]:
# 학습횟수를 카운트하는 변수정의 
global_step=tf.Variable(0,trainable=False,name='global_step') #학습횟수 카운트를 위해 trainable이라는 옵션을 줌

- 앞장에서보다 계층을 하나 늘리고
- 편향은 없이 가중치만 사용한 모델로 만듬

In [5]:
X=tf.placeholder(tf.float32)
Y=tf.placeholder(tf.float32)

W1=tf.Variable(tf.random_uniform([2,10],-1,1))
L1=tf.nn.relu(tf.matmul(X,W1))

W2=tf.Variable(tf.random_uniform([10,20],-1,1))
L2=tf.nn.relu(tf.matmul(L1,W2))

W3=tf.Variable(tf.random_uniform([20,3],-1,1))
model=tf.matmul(L2,W3)

cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,logits=model))
optimizer=tf.train.AdamOptimizer(learning_rate=0.01)
train_op=optimizer.minimize(cost,global_step=global_step)#모델을 최적화 할때마다 global_step의 변수의 값을 1씩 증가함

In [24]:
sess=tf.Session()
saver=tf.compat.v1.train.Saver(tf.global_variables()) # 앞서 정의한 변수들을 가져오는 함수 (이전에 학습한 결과를 불러와 저장)

#### 모델 재사용
- 기존에 학습해둔 모델이 있는지 확인 후 있다면 saver.restore함수를 이용해 학습된 값을 불러오고 아니면 변수를 새로 초기화
- 학습한 모델을 저장한 파일 = checkpoint file

In [25]:
ckpt=tf.train.get_checkpoint_state('./model') # checkpoint file이 있는지 확인 > 있다면 checkpoint state, 없다면 None반환
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess,ckpt.model_checkpoint_path)#ck 파일 불러와서 변수에 할당.
else:
    sess.run(tf.global_variables_initializer()) # 없으면 초기화

In [26]:
#global_step은 텐서타입의 변수이므로 값을 가져올때 sess.run(global_step)을 이용해야 한다.
for step in range(2):
    sess.run(train_op,feed_dict={X:x_data,Y:y_data})
    
    print('step %d'%sess.run(global_step),
         'cost %.3f'%sess.run(cost,feed_dict={X:x_data,Y:y_data}))

# 최적화 완료 후 학습된 변수들을 지정한 chpt파일에 저장
saver.save(sess,'./model/dnn.ckpt',global_step=global_step)

step 1 cost 2.339
step 2 cost 2.185


'./model/dnn.ckpt-2'

## 2. 텐서보드 사용하기